In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
import ariel_load
import tqdm
importlib.reload(kgs);

local
local


In [2]:
%%time
train_data = kgs.load_all_train_data()
test_data = kgs.load_all_test_data()
len(train_data), len(test_data)
data = train_data[0]

CPU times: user 472 ms, sys: 80.1 ms, total: 552 ms
Wall time: 1.64 s


In [3]:
def robust_linear_fit_rms(X, Y, drop_frac=0.05):
    X = np.asarray(X)
    Y = np.asarray(Y)

    # Fit Y = aX + b using least squares
    A = np.vstack([X, np.ones_like(X)]).T
    a, b = np.linalg.lstsq(A, Y, rcond=None)[0]

    # Predicted Y and residuals
    Y_pred = a * X + b
    residuals = Y - Y_pred
    abs_resid = np.abs(residuals)

    # Remove worst 5% (highest absolute residuals)
    if drop_frac>0:
        n = len(residuals)
        keep = int(n * (1 - drop_frac))
        indices = np.argpartition(abs_resid, keep)[:keep]
    #print(keep, len(residuals))
    else:
        indices = np.arange(len(residuals))

    # Compute RMS of the remaining residuals
    rms = np.sqrt(np.mean(residuals[indices] ** 2))
    print(a,b)
    return rms

In [ ]:
import ariel_simple
importlib.reload(ariel_simple)
df = 0.05
for jj in range(2):
    model = ariel_simple.SimpleModel()
    model.run_in_parallel = False
    match jj:
        case 0:
            name = 'Default'
        case 1:
            name = 'Use correction factor'
            model.use_correction_factor = True        
    model.do_plots = False
    model.train(train_data)
    data = train_data
    inferred_data = model.infer(data)
    print(name)
    solution = kgs.make_submission_dataframe(data, include_sigma=False)
    submission = kgs.make_submission_dataframe(inferred_data, False)
    print(1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=df), 1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=0))
    print(1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=df),
         1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=0))

Inferring:   5%|███▌                                                                  | 56/1100 [02:53<52:47,  3.03s/it]

In [ ]:
print(1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=df), 1e6*robust_linear_fit_rms(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy(), drop_frac=0))
print(1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=df),
    1e6*robust_linear_fit_rms(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1),drop_frac=0))

In [ ]:
score, rms_err = kgs.score_metric(inferred_data, data)
#np.sqrt(np.diag(inferred_data[0].spectrum_cov))

In [ ]:
import copy
data_sim = copy.deepcopy(data)
sig = rms_err
for ii,d in enumerate(data_sim):
    d.spectrum += np.random.default_rng(seed=ii).normal(0,sig,size=d.spectrum.shape)
    d.spectrum_cov = np.diag(0*d.spectrum+sig**2)
    d.check_constraints()
print('Theoretical')
kgs.score_metric(data_sim, data);

In [ ]:
kgs.print_sanity_checks()

In [ ]:
plt.figure()
plt.plot(kgs.sanity_checks['simple_residual_ratio_FGS'].seen_all)
plt.plot(kgs.sanity_checks['simple_residual_ratio_AIRS'].seen_all)
plt.legend(('FGS', 'AIRS'))
plt.grid(True)

In [ ]:
solution = kgs.make_submission_dataframe(data, include_sigma=False)
submission = kgs.make_submission_dataframe(inferred_data, False)


In [ ]:
kgs.rms(solution.iloc[:,1:2].to_numpy()-submission.iloc[:,1:2].to_numpy()),kgs.rms(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy())

In [ ]:
_,ax=plt.subplots(1,2,figsize=(10,5))
plt.sca(ax[0])
plt.box(True);plt.grid(True);
plt.scatter(solution.iloc[:,1].to_numpy(), solution.iloc[:,1].to_numpy()-submission.iloc[:,1].to_numpy())
plt.title(1e6*kgs.rms(solution.iloc[:,1:2].to_numpy()-submission.iloc[:,1:2].to_numpy()))
plt.sca(ax[1])
plt.box(True);plt.grid(True);
plt.scatter(np.mean(solution.iloc[:,2:284].to_numpy(),1), np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1))
plt.title(1e6*kgs.rms(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy()))

In [ ]:
ind = np.argmax(np.abs(np.mean(solution.iloc[:,2:284].to_numpy()-submission.iloc[:,2:284].to_numpy(),1)))
ind

In [ ]:
import ariel_simple
import copy
kgs.sanity_checks_active = False
importlib.reload(ariel_simple)
ind=968
model = ariel_simple.SimpleModel()
model.run_in_parallel = False
model.do_plots = True
model.use_correction_factor = True
model.train(train_data)
data = copy.deepcopy(train_data[ind:ind+1])
#kgs.list_attrs(data[0])
# for d_ref in train_data:
#     dat = copy.deepcopy(d_ref)
#     dat.spectrum = train_data[ind].spectrum
#     dat.planet_id = train_data[ind].planet_id
#     data = [dat]
    #data[0].sma = sma
    #for ii in range(2):
    #    pass
       #model.loaders[ii].apply_full_sensor_corrections.remove_background_based_on_pixels = False    
       #model.loaders[ii].apply_full_sensor_corrections.remove_background_based_on_rows = False    
     #   model.loaders[ii].apply_pixel_corrections.mask_hot = False
     #   model.loaders[ii].apply_pixel_corrections.mask_hot = False
inferred_data = model.infer(data)
print(model.correction_factor)
print(np.mean(data[0].spectrum[:1]-inferred_data[0].spectrum[:1]), np.mean(data[0].spectrum[1:]-inferred_data[0].spectrum[1:]), kgs.rms(model.pred[1]-model._targets[1]))

In [ ]:
%%time
import ariel_load
import copy
kgs.profiling=False
importlib.reload(ariel_load)
loaders = ariel_load.default_loaders()
d = copy.deepcopy(train_data[968])
d.transits[0].load_to_step(2, d, loaders)
print(d.transits[0].data[0].data.shape, d.transits[0].data[1].data.shape, d.transits[0].data[0].data[:2,:2,:2], d.transits[0].data[1].data[:2,:2,:2])
plt.figure()
plt.imshow(np.log(np.mean(d.transits[0].data[0].data.get(), axis=0)),aspect='auto')
plt.colorbar()
plt.figure()
plt.imshow(np.log(np.mean(d.transits[0].data[1].data.get(), axis=0)),aspect='auto')
plt.colorbar()
#d.transits[0].load_to_step(0,d,loaders)
#d.transits[0].data[0]
#ref = kgs.dill_load(kgs.temp_dir + '/ref.pickle')
#assert cp.array_equal(ref[0].data, d.transits[0].data[0].data, equal_nan=True)
#assert cp.array_equal(ref[1].data, d.transits[0].data[1].data, equal_nan=True)

In [ ]:
%%time
import ariel_load
import copy
kgs.profiling=False
importlib.reload(ariel_load)
loaders = ariel_load.default_loaders()
d = copy.deepcopy(train_data[0])
d.transits[0].load_to_step(2, d, loaders)
print(d.transits[0].data[0].data.shape, d.transits[0].data[1].data.shape, d.transits[0].data[0].data[:2,:2,:2], d.transits[0].data[1].data[:2,:2,:2])
plt.figure()
plt.imshow(np.log(np.mean(d.transits[0].data[0].data.get(), axis=0)),aspect='auto')
plt.colorbar()
plt.figure()
plt.imshow(np.log(np.mean(d.transits[0].data[1].data.get(), axis=0)),aspect='auto')
plt.colorbar()
#d.transits[0].load_to_step(0,d,loaders)
#d.transits[0].data[0]
#ref = kgs.dill_load(kgs.temp_dir + '/ref.pickle')
#assert cp.array_equal(ref[0].data, d.transits[0].data[0].data, equal_nan=True)
#assert cp.array_equal(ref[1].data, d.transits[0].data[1].data, equal_nan=True)